In [ ]:
#Only needed for colab
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#Only needed for colab
import condacolab
condacolab.check()


In [ ]:
#only needed for colab
!mamba install -c conda-forge "hoomd=4.7.0" vim openssh mbuild gsd ipython ipykernel mdanalysis foyer freud ipywidgets scipy numpy physical_validation matplotlib jupyter pandas flowermd nglview mdanalysis 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Imports
from cmeutils.gsd_utils import get_molecule_cluster
import freud
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np

# flowerMD: https://github.com/cmelab/flowerMD
import flowermd
from flowermd.base import System, Simulation
from flowermd.library.polymers import PolyEthylene, LJChain
from flowermd.library.forcefields import GAFF, BeadSpring

In [ ]:
def radius_of_gyration(gsd_file, start=0, stop=-1, stride=1):
    """Calculates the radius of gyration of a molecule using Freud's cluster module.

    Freud: https://freud.readthedocs.io/en/latest/

    Notes
    -----
    This method is designed to create a time series of the radius of gyraiton measurement
    of a single polymer chain simulation.

    Parameters
    ----------
    gsd_file : str; required
        Path to a gsd_file
    start: int; optional; default 0
        The frame index of the trajectory to begin with
    stop: int; optional; default -1
        The frame index of the trajectory to end with
    stride: int, optional, default 1
        The stride (i.e. spacing between samples) to use between start and stop

    Returns
    -------
    rg_values : List of arrays of floats
        Array of individual chain Rg values for each frame
        
    """
    trajectory = gsd.hoomd.open(gsd_file, mode="r")
    rg_values = []

    for snap in trajectory[start:stop:stride]:
        system = freud.AABBQuery.from_system(snap)
        n_query_points = n_points = snap.particles.N
        query_point_indices = snap.bonds.group[:, 0]
        point_indices = snap.bonds.group[:, 1]
        box = freud.box.Box(
            snap.configuration.box[0],
            snap.configuration.box[1],
            snap.configuration.box[2],
        )
        vectors = box.wrap(
            snap.particles.position[query_point_indices]
            - snap.particles.position[point_indices]
        )
        nlist = freud.NeighborList.from_arrays(
            num_query_points=n_query_points,
            num_points=n_points,
            query_point_indices=query_point_indices,
            point_indices=point_indices,
            vectors=vectors,
        )
        cluster = freud.cluster.Cluster()
        cluster.compute(system=system, neighbors=nlist)
        cl_props = freud.cluster.ClusterProperties()
        cl_props.compute(system, cluster.cluster_idx)       
        rg_values.append(cl_props.radii_of_gyration)
    return rg_values


class SingleChainSystem(System):
    """We'll make our own class in flowerMD which inherits from flowermd.base.System

    This class is designed to make a box with edges equal to the polymer backbone length
    and place the polymer chain in the center of the box.
    
    """
    def __init__(self, molecules, base_units=dict()):
        super(SingleChainSystem, self).__init__(
                molecules=molecules,
                base_units=base_units
        )

    def _build_system(self):
        chain = self.all_molecules[0]
        head = chain.children[0]
        tail = chain.children[-1]
        chain_length = np.linalg.norm(tail.pos - head.pos)
        box = mb.Box(lengths=np.array([chain_length] * 3) * 1.05)
        comp = mb.Compound()
        comp.add(chain)
        comp.box = box
        chain.translate_to((box.Lx / 2, box.Ly / 2, box.Lz / 2))
        return comp

# Running a single chain simulation in a vacuum (i.e. very low density)

### Building the initial configuraiton and forces

In [ ]:
# Define the chain length for your single-chain simulation
chain_length = 20 #For interactive demos, start between 15-30

chains = LJChain(
    num_mols=1,
    lengths=chain_length,
    bond_lengths={"A-A": 1.12}
)
single_chain_system = SingleChainSystem(molecules=chains)
forces = BeadSpring(
    r_cut=2.5,
    beads={"A": dict(epsilon=1.0, sigma=1.0)},
    bonds={"A-A": dict(k=200, r0=1.12)},
    angles={"A-A-A": dict(k=100, t0=2.2)}
)

### Running a simulation with our initial configuration and forces:

In [ ]:
##### Define these parameters for your simulation ####
n_steps = 1e6
kT = 3.0

gsd_write_frequency = n_steps // 100
log_write_freqeuncy = n_steps // 100
gsd_file_name = f"{chain_length}chain_length.gsd"
log_file_name = f"{chain_length}chain_length.txt"

sim = Simulation(
    initial_state=single_chain_system.hoomd_snapshot,
    forcefield=forces.hoomd_forces,
    gsd_write_freq=gsd_write_frequency,
    log_write_freq=log_write_freqeuncy,
    gsd_file_name=gsd_file_name,
    log_file_name=log_file_name,
    dt=0.0008
)

In [ ]:
sim.run_NVT(n_steps=n_steps, kT=kT, tau_kt=sim.dt * 100)
sim.flush_writers()

### Measuring the radius of gyration over a range of simulation snapshots

In [ ]:
rg_values = radius_of_gyration(gsd_file=gsd_file_name, start=-30, stop=-1, stride=2)

In [ ]:
rg_avg = np.mean(rg_values)
rg_error = np.std(rg_values)

In [ ]:
print(rg_avg, rg_error)

# Running single chain simulations over several chain lengths

### We'll make a list of chain lengths and use a for loop to build a system and run a simulation for each length

In [ ]:
# Define your simulation parameters here

n_steps = 1e6
kT = 4.0
num_frames = 500
#chain_lengths = [25, 50, 75, 100, 150, 200, 300, 400]
chain_lengths = [25, 50, 100, 200, 400]

gsd_write_frequency = n_steps // num_frames
log_write_freqeuncy = n_steps // num_frames

for chain_length in chain_lengths:
    print(f"Starting a new simulation of chain length {chain_length}")
    print("==========================================================")
    chains = LJChain(
        num_mols=1,
        lengths=chain_length,
        bond_lengths={"A-A": 1.12}
    )
    single_chain_system = SingleChainSystem(molecules=chains)
    forces = BeadSpring(
        r_cut=2.5,
        beads={"A": dict(epsilon=1.0, sigma=1.0)},
        bonds={"A-A": dict(k=200, r0=1.12)},
        angles={"A-A-A": dict(k=100, t0=2.2)}
    )

    gsd_file_name = f"{chain_length}chain_length.gsd"
    log_file_name = f"{chain_length}chain_length.txt"
    
    sim = Simulation(
        initial_state=single_chain_system.hoomd_snapshot,
        forcefield=forces.hoomd_forces,
        gsd_write_freq=gsd_write_frequency,
        log_write_freq=log_write_freqeuncy,
        gsd_file_name=gsd_file_name,
        log_file_name=log_file_name,
        dt=0.0008
    )
    sim.run_NVT(n_steps=n_steps, kT=kT, tau_kt=sim.dt * 100)
    sim.flush_writers()

### Plot the results for each chain length

In [ ]:
fig = plt.figure()

for chain_length in chain_lengths:
    gsd_file_name = f"{chain_length}chain_length.gsd"
    rg_values = radius_of_gyration(gsd_file=gsd_file_name, start=250, stop=-1, stride=3)
    plt.errorbar(x=chain_length, y=np.mean(rg_values), yerr=np.std(rg_values), color="k", marker="o")

plt.xlabel("N beads")
plt.ylabel("$R_g (\sigma)$")